# CalcGPT: Building an Arithmetic Language Model from Scratch

**A Complete Guide to Transformer-Based Language Models using HuggingFace and PyTorch**

---

## 🎯 Overview

Welcome to **CalcGPT** - a comprehensive tutorial on building, training, and deploying transformer-based language models for arithmetic tasks. This notebook demonstrates the complete machine learning pipeline from dataset generation to production inference, while teaching fundamental concepts of modern NLP.

**🔥 UPDATED**: This notebook now uses the **CalcGPT Library** (`lib/` package) for programmatic access, demonstrating both library usage and CLI tools!

### 🌟 What You'll Learn

- **Transformer Architecture**: Understanding GPT-2 models and attention mechanisms
- **Dataset Engineering**: Creating and analyzing training datasets for language models
- **Model Training**: End-to-end training with HuggingFace Transformers
- **Evaluation Methodologies**: Comprehensive model assessment and validation
- **Production Deployment**: Interactive inference and real-world usage
- **Scaling Strategies**: From toy models to production-ready systems
- **Library Integration**: Using CalcGPT as both a library and CLI tool

### 🛠️ Tools We'll Use

- **CalcGPT Library** (`lib/`): Programmatic access to all functionality
  - `DatasetGenerator` & `DatagenConfig`: Dataset generation
  - `CalcGPTTrainer` & `TrainingConfig`: Model training  
  - `CalcGPT` & `InferenceConfig`: Model inference
  - `CalcGPTEvaluator` & `EvaluationConfig`: Model evaluation
- **CalcGPT CLI Tools**: Interactive command-line interfaces
  - `calcgpt_dategen.py`: Dataset generation tool
  - `calcgpt_train.py`: Model training tool
  - `calcgpt_eval.py`: Model evaluation tool
  - `calcgpt.py`: Interactive inference tool

### 📚 Learning Path

1. **Simple Start**: Basic arithmetic with tiny models (38K parameters) using the library
2. **Understanding**: Deep dive into model architecture and training dynamics
3. **Scaling Up**: Larger datasets and models (1.2M+ parameters) programmatically
4. **Production**: Real-world inference and deployment with both library and CLI

Let's build something amazing! 🚀


## 🔧 Setup and Imports

First, let's import all the necessary libraries and set up our environment. We'll be using modern PyTorch and HuggingFace transformers throughout this tutorial.


In [2]:
# Core libraries
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import time
from datetime import datetime
import subprocess
import sys

# HuggingFace transformers
from transformers import GPT2Config, GPT2LMHeadModel, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

# CalcGPT Library - Our new programmatic interface!
from lib import (
    DatasetGenerator, DatagenConfig,
    CalcGPTTrainer, TrainingConfig, 
    CalcGPT, InferenceConfig,
    CalcGPTEvaluator, EvaluationConfig
)

# Utility imports
import warnings
warnings.filterwarnings('ignore')

# Set style for beautiful plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Check available devices
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f"🎯 Using device: {device}")
print(f"🐍 Python version: {sys.version}")
print(f"🔥 PyTorch version: {torch.__version__}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("✅ Setup complete! Ready to build CalcGPT 🚀")


/Users/mihai/Coding/jupyter/transformers/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🎯 Using device: mps
🐍 Python version: 3.13.4 (main, Jun  3 2025, 15:34:24) [Clang 17.0.0 (clang-1700.0.13.3)]
🔥 PyTorch version: 2.7.1
✅ Setup complete! Ready to build CalcGPT 🚀


## 📊 Part 1: Understanding the Problem & Dataset Generation

### The Challenge: Teaching Machines Arithmetic

Language models like GPT-3 can write poetry and code, but struggle with basic arithmetic. Why? Because arithmetic requires **precise computation** rather than **pattern matching**. This makes arithmetic an excellent testbed for understanding model capabilities and limitations.

### Our Approach: Character-Level Language Modeling

We'll treat arithmetic as a **sequence-to-sequence** problem:
- **Input**: `"1+1="` 
- **Target**: `"1+1=2"`

The model learns to predict the next character given the previous characters, eventually learning to compute arithmetic results.

### Dataset Design Philosophy

Our CalcGPT DataGen tool creates intelligent datasets with:
- **Systematic coverage**: All combinations within specified ranges
- **Data augmentation**: Commutative property examples (a+b and b+a)
- **Intelligent naming**: Filenames encode generation parameters
- **Scalability**: From toy problems to complex arithmetic

Let's start by generating a simple dataset for our first model!


In [3]:
# Generate a simple dataset for our first model using the CalcGPT library
# We'll start small: numbers 0-5, only addition, limit to 20 examples

print("🎬 Generating simple dataset with CalcGPT DatasetGenerator...")

# Create configuration for simple dataset
simple_config = DatagenConfig(
    max_value=5,                    # Max value: 5
    max_expressions=20,             # Limit: 20 examples
    operations=['addition'],        # Addition only
    verbose=True
)

# Generate dataset programmatically
generator = DatasetGenerator(simple_config)
dataset_path = generator.generate()

print(f"✅ Dataset generated at: {dataset_path}")

# Load and analyze the generated dataset
simple_dataset = generator.load_dataset(dataset_path)

print(f"\n📚 Generated dataset preview:")
print(f"Total examples: {len(simple_dataset)}")
print("First 10 examples:")
for i, example in enumerate(simple_dataset[:10]):
    print(f"  {i+1:2d}. {example}")

if len(simple_dataset) > 10:
    print("  ...")
    print(f"  {len(simple_dataset)}. {simple_dataset[-1]}")

# Analyze the dataset using our programmatic interface
analysis = generator.analyze_dataset(simple_dataset)
print(f"\n📊 Dataset Analysis:")
print(f"  📏 Average length: {analysis['avg_length']:.1f} characters")
print(f"  📏 Max length: {analysis['max_length']} characters")
print(f"  🔤 Unique characters: {analysis['vocabulary']}")
print(f"  📈 Character count: {analysis['vocab_size']} unique chars")
print(f"  📊 Operations: {analysis['operations']}")


🎬 Generating simple dataset with CalcGPT DataGen...
STDOUT:


╔═══════════════════════════════════════════════════════════════╗
║                    CalcGPT DataGen                            ║
║                 Dataset Generation Tool                      ║
║                         v1.0.0                               ║
╚═══════════════════════════════════════════════════════════════╝


🚀 Generation Configuration:
  🎯 Value range: 0 - 5
  🔢 Allowed digits: All digits (0-9)
  🧮 Operations: ➕ addition
  📏 Expression limit: 20
  📁 Output file: datasets/ds-calcgpt_min0_max5_alldigits_add_limit20.txt

🎬 Starting expression generation...
📝 Writing expressions to: datasets/ds-calcgpt_min0_max5_alldigits_add_limit20.txt
🧮 Generating arithmetic expressions...
🔢 Generating valid numbers up to 5...
✅ Generated 6 numbers (all digits allowed)
🔧 Operations to include: addition
📊 Estimated expressions to generate: ~36
⚠️ Reached maximum expression limit: 20
✅ Successfully wrote 20 expressions in 0.

## 🧠 Part 2: Understanding Transformer Architecture

### The GPT-2 Architecture

Our CalcGPT is based on **GPT-2** (Generative Pre-trained Transformer), which uses the **decoder-only** transformer architecture. Let's understand the key components:

#### 🔧 Key Components

1. **Token Embeddings**: Convert characters to dense vectors
2. **Positional Embeddings**: Encode position information
3. **Multi-Head Attention**: Learn relationships between positions
4. **Feed-Forward Networks**: Non-linear transformations
5. **Layer Normalization**: Stabilize training
6. **Causal Masking**: Prevent future token access

#### 📐 Model Parameters

For our simple model, we'll use a tiny architecture:
- **Embedding dimension**: 32 (vs 768 in GPT-2 small)
- **Number of layers**: 1 (vs 12 in GPT-2 small)
- **Attention heads**: 2 (vs 12 in GPT-2 small)
- **Vocabulary size**: ~7 characters (`0123456789+=`)

This gives us only ~38K parameters vs 117M in GPT-2 small!

#### 🎯 Training Objective

**Causal Language Modeling**: Given a sequence `x₁, x₂, ..., xₙ`, predict `xₙ₊₁`

For `"1+1=2"`:
- Input: `"1+1="` → Predict: `"2"`
- The model learns: `P(2|1,+,1,=)`

### Why Start Small?

1. **Fast iteration**: Quick training and testing
2. **Understanding**: Easier to analyze and debug
3. **Resource efficiency**: Runs on any hardware
4. **Clear baselines**: Establish performance expectations

Let's train our first tiny CalcGPT model!


In [4]:
# Train our first tiny CalcGPT model using the library
print("🚀 Training tiny CalcGPT model with CalcGPTTrainer...")

# Create training configuration for tiny model
tiny_config = TrainingConfig(
    epochs=3,               # Quick training
    batch_size=4,           # Small batches  
    learning_rate=1e-3,     # Default learning rate
    embedding_dim=32,       # Small embedding
    num_layers=1,           # Single layer
    num_heads=2,            # Two attention heads
    test_split=0.0,         # No validation for simplicity
    verbose=True
)

# Train the model programmatically
training_start = time.time()

trainer = CalcGPTTrainer(
    config=tiny_config,
    dataset_path=dataset_path,  # Use our generated dataset
    output_dir=Path('models/tiny_calcgpt'),
    verbose=True
)

# Train and get results
results = trainer.train()
training_time = time.time() - training_start

print(f"\n✅ Training completed in {training_time:.1f} seconds")

# Display training results
print(f"\n📊 Training Results:")
print(f"  📈 Final loss: {results['training_loss']:.4f}")
print(f"  ⏱️  Training time: {results['training_time']/60:.2f} minutes")
print(f"  🧠 Model parameters: {results['model_params']:,}")
print(f"  📚 Dataset size: {results['dataset_size']:,} examples")
print(f"  🔤 Vocabulary size: {results['vocab_size']} tokens")

print(f"\n🧪 Quick Test Results:")
for prompt, result in results['test_results'].items():
    print(f"  {prompt} → {result}")

print(f"\n📁 Model saved to: {trainer.output_dir}")

# Store the model path for later use
tiny_model_path = trainer.output_dir


🚀 Training tiny CalcGPT model with our professional trainer...
STDOUT:


╔═══════════════════════════════════════════════════════════════╗
║                    CalcGPT Trainer                            ║
║              Advanced Model Training System                   ║
║                         v1.0.0                               ║
╚═══════════════════════════════════════════════════════════════╝

🍎 Apple Silicon (MPS) detected
📚 Loading dataset from: datasets/ds-calcgpt_min0_max5_alldigits_add_limit20.txt
✅ Loaded 20 examples from dataset
📊 Dataset statistics:
   Average length: 5.0 characters
   Maximum length: 5 characters
   Minimum length: 5 characters
✨ Applying data augmentation (commutative property)...
✅ Added 7 augmented examples
📈 Total dataset size: 27 examples
🔤 Creating optimized vocabulary...
✅ Vocabulary created with 12 tokens
🔧 Special tokens: ['<pad>', '<eos>']
🔧 Character tokens: +01234567=
📏 Maximum sequence length: 6
⚠️ No validation split (evaluation disabled)



## 📊 Part 3: Model Evaluation and Analysis

### Comprehensive Evaluation Strategy

Now let's evaluate our tiny model using CalcGPT Eval. This tool provides comprehensive assessment across multiple dimensions:

#### 🧪 Test Types
1. **First Operand**: Given `"1"`, can it complete to `"1+0=1"`?
2. **Expression Complete**: Given `"1+1"`, can it add `"=2"`?
3. **Answer Complete**: Given `"1+1="`, can it predict `"2"`?

#### 📏 Metrics
- **Format Validity**: Does output follow `num+num=num` pattern?
- **Arithmetic Correctness**: Is the math actually correct?
- **Completion Success**: Does the model generate complete expressions?
- **Performance Timing**: How fast is inference?

Let's see how our tiny model performs!


In [5]:
# Evaluate our tiny model using the library
print("📊 Evaluating tiny CalcGPT model...")

# Create evaluation configuration
eval_config = EvaluationConfig(
    sample_size=30,          # Test on 30 cases
    max_tokens=10,           # Allow up to 10 tokens for completion
    verbose=True
)

# Initialize evaluator with our trained model
evaluator = CalcGPTEvaluator(
    config=eval_config,
    model_path=tiny_model_path,  # Use our trained model
    dataset_path=dataset_path,   # Same dataset we trained on
    verbose=True
)

# Run comprehensive evaluation
eval_results = evaluator.evaluate()

# Display evaluation results
print(f"\n📊 Evaluation Results:")
print(f"  🎯 Overall accuracy: {eval_results['accuracy_stats']['overall']:.1%}")
print(f"  ✅ Format validity: {eval_results['accuracy_stats']['format']:.1%}")
print(f"  🧮 Arithmetic correctness: {eval_results['accuracy_stats']['arithmetic']:.1%}")
print(f"  📝 Complete expressions: {eval_results['accuracy_stats']['complete']:.1%}")

print(f"\n📈 Performance by Test Type:")
for test_type, stats in eval_results['test_type_stats'].items():
    print(f"  {test_type.replace('_', ' ').title()}:")
    print(f"    Arithmetic: {stats['arithmetic']:.1%}")
    print(f"    Format: {stats['format']:.1%}")

print(f"\n⏱️ Performance Timing:")
timing = eval_results['timing_stats']
print(f"  Mean: {timing['mean']:.1f}ms")
print(f"  Median: {timing['median']:.1f}ms")
print(f"  Range: {timing['min']:.1f}ms - {timing['max']:.1f}ms")

# Also try some manual inference using the CalcGPT class for comparison
print("\n" + "="*60)
print("🔍 PROGRAMMATIC INFERENCE ANALYSIS")
print("="*60)

# Initialize inference model
inference_config = InferenceConfig(
    temperature=0.0,  # Deterministic inference
    max_tokens=10,
    verbose=False
)

calc_model = CalcGPT(
    config=inference_config,
    model_path=tiny_model_path,
    verbose=False
)

# Test problems
test_problems = ["1+1=", "2+0=", "0+2=", "3+1=", "2+2="]

print("Problem     → Predicted  (Expected)  Status")
print("-" * 45)

for problem in test_problems:
    try:
        result = calc_model.generate(problem)
        predicted = result['completion'].strip()
        
        # Extract operands and calculate expected
        expr = problem.replace('=', '')
        if '+' in expr:
            operands = expr.split('+')
            expected = int(operands[0]) + int(operands[1])
        else:
            expected = "?"
        
        # Check if prediction matches expected
        is_correct = str(predicted) == str(expected)
        status = "✅" if is_correct else "❌"
        
        print(f"{problem:10s} → {predicted:9s}  ({expected:8s})  {status}")
        
    except Exception as e:
        print(f"{problem:10s} → ERROR     (?)        ❌")

print(f"\n💡 The tiny model shows the learning process - it's beginning to understand")
print(f"   the task structure but needs more capacity and training for accuracy!")


📊 Evaluating tiny CalcGPT model...
STDOUT:


╔═══════════════════════════════════════════════════════════════╗
║                        CalcGPT Eval                          ║
║                   Model Evaluation Tool                      ║
║                         v1.0.0                               ║
╚═══════════════════════════════════════════════════════════════╝

🎯 Auto-detected model: calcgpt_emb32_lay1_head2_ep3_bs4_lr1e03_ds20
Initializing CalcGPT evaluator...
Loading model from: models/calcgpt_emb32_lay1_head2_ep3_bs4_lr1e03_ds20
Using checkpoint: models/calcgpt_emb32_lay1_head2_ep3_bs4_lr1e03_ds20/checkpoint-12
✅ Model loaded successfully!
   Parameters: 38,624
   Device: mps
✅ Vocabulary loaded:
   Vocab size: 7
   Max length: 15
   Vocabulary: {'<pad>': 0, '<eos>': 1, '+': 2, '0': 3, '1': 4, '2': 5, '=': 6}
Loading evaluation dataset: datasets/ds-calcgpt_min0_max5_alldigits_add_limit20.txt
✅ Loaded 20 equations from dataset
✅ Generated 60 test cases
📝 Using random sample 

## 🎯 Part 4: Scaling Up - Production-Ready CalcGPT

### What We Learned from Our Tiny Model

Our 38K parameter model taught us valuable lessons:

1. **Architecture Matters**: Even tiny transformers can learn patterns
2. **Data Quality > Quantity**: Small, clean datasets can be effective
3. **Evaluation is Critical**: Multiple test types reveal different capabilities
4. **Training Dynamics**: Fast convergence on simple problems

### Limitations of the Tiny Model

- **Limited Capacity**: Can't handle complex arithmetic
- **Poor Generalization**: Struggles with unseen number combinations
- **Format Issues**: May not always produce valid expressions
- **Narrow Range**: Only works within training data distribution

### Scaling Strategy

Now let's build a **production-ready** CalcGPT with:

#### 📈 Larger Dataset
- **Range**: Numbers 0-100 (vs 0-5)
- **Operations**: Both addition and subtraction
- **Size**: ~10,000+ examples (vs 20)
- **Augmentation**: Commutative examples included

#### 🏗️ Bigger Architecture
- **Embedding Dimension**: 128 (vs 32)
- **Layers**: 6 (vs 1) 
- **Attention Heads**: 8 (vs 2)
- **Parameters**: ~1.2M (vs 38K)

#### ⚡ Advanced Training
- **Validation Split**: Proper train/test separation
- **Learning Rate Scheduling**: Cosine annealing
- **Early Stopping**: Based on validation loss
- **Mixed Precision**: Faster training where available

Let's build the real deal! 🚀


In [6]:
# Generate a comprehensive dataset for production CalcGPT using the library
print("🎬 Generating comprehensive dataset for production model...")

# Create configuration for production dataset
production_config = DatagenConfig(
    max_value=100,               # Max value: 100 (much larger!)
    operations=['addition', 'subtraction'],  # Both operations
    verbose=True
)

# Generate dataset programmatically
generation_start = time.time()
production_generator = DatasetGenerator(production_config)
production_dataset_path = production_generator.generate()
generation_time = time.time() - generation_start

print(f"✅ Production dataset generated at: {production_dataset_path}")
print(f"⏱️ Dataset generation completed in {generation_time:.1f} seconds")

# Load and analyze the comprehensive dataset
full_dataset = production_generator.load_dataset(production_dataset_path)
analysis = production_generator.analyze_dataset(full_dataset)

print(f"\n📚 Production Dataset Analysis:")
print(f"  📁 File: {Path(production_dataset_path).name}")
print(f"  📊 Total examples: {len(full_dataset):,}")
print(f"  📏 Average length: {analysis['avg_length']:.1f} characters")
print(f"  📏 Max length: {analysis['max_length']} characters")
print(f"  🔤 Vocabulary size: {analysis['vocab_size']} characters")
print(f"  💾 File size: {Path(production_dataset_path).stat().st_size / 1024:.1f} KB")

# Show some examples from different ranges
print(f"\n📋 Sample expressions:")
examples_to_show = [0, len(full_dataset)//4, len(full_dataset)//2, -1]
for i in examples_to_show:
    if i < len(full_dataset):
        print(f"  {full_dataset[i]}")

# Analyze the distribution of operations using our analysis
print(f"\n📊 Operation distribution:")
for op, count in analysis['operations'].items():
    percentage = count / len(full_dataset) * 100
    op_symbol = "➕" if op == "addition" else "➖"
    print(f"  {op_symbol} {op.title()}: {count:,} ({percentage:.1f}%)")

print(f"\n🎯 Ready for production training with: {Path(production_dataset_path).name}")

# Store the dataset path for training
production_dataset = production_dataset_path


🎬 Generating comprehensive dataset for production model...
STDOUT:


╔═══════════════════════════════════════════════════════════════╗
║                    CalcGPT DataGen                            ║
║                 Dataset Generation Tool                      ║
║                         v1.0.0                               ║
╚═══════════════════════════════════════════════════════════════╝


🚀 Generation Configuration:
  🎯 Value range: 0 - 100
  🔢 Allowed digits: All digits (0-9)
  🧮 Operations: ➕ addition and ➖ subtraction
  📏 Expression limit: Unlimited
  📁 Output file: datasets/ds-calcgpt_min0_max100_alldigits_allops.txt

🎬 Starting expression generation...
📝 Writing expressions to: datasets/ds-calcgpt_min0_max100_alldigits_allops.txt
🧮 Generating arithmetic expressions...
🔢 Generating valid numbers up to 100...
✅ Generated 101 numbers (all digits allowed)
🔧 Operations to include: addition, subtraction
📊 Estimated expressions to generate: ~15,301
   Generated 1,000 expressions..

In [7]:
# Train the production CalcGPT model using the library
print("🚀 Training production CalcGPT model...")
print("⚠️ This will take longer but results in much better performance!")

# Create production training configuration
production_config = TrainingConfig(
    epochs=20,              # More training
    batch_size=8,           # Reasonable batch size
    learning_rate=1e-3,     # Default learning rate
    embedding_dim=128,      # Larger embeddings
    num_layers=6,           # Deeper network
    num_heads=8,            # More attention heads
    test_split=0.2,         # Proper validation split
    save_steps=500,         # Save checkpoints
    verbose=True
)

# Train the production model programmatically
production_training_start = time.time()

production_trainer = CalcGPTTrainer(
    config=production_config,
    dataset_path=production_dataset,    # Our comprehensive dataset
    output_dir=Path('models/production_calcgpt'),
    verbose=True
)

# Train and get results (this will take several minutes)
production_results = production_trainer.train()
production_training_time = time.time() - production_training_start

print(f"\n✅ Production training completed in {production_training_time/60:.1f} minutes")

# Display comprehensive training results
print(f"\n📊 Production Training Results:")
print(f"  📈 Final training loss: {production_results['training_loss']:.4f}")
if production_results['eval_loss']:
    print(f"  📉 Validation loss: {production_results['eval_loss']:.4f}")
print(f"  ⏱️  Training time: {production_results['training_time']/60:.1f} minutes")
print(f"  🧠 Model parameters: {production_results['model_params']:,}")
print(f"  📚 Dataset size: {production_results['dataset_size']:,} examples")
print(f"  🔤 Vocabulary size: {production_results['vocab_size']} tokens")

print(f"\n🧪 Production Test Results:")
for prompt, result in production_results['test_results'].items():
    print(f"  {prompt} → {result}")

# Analyze model size
model_files = list(production_trainer.output_dir.rglob('*.bin'))
if model_files:
    total_size = sum(f.stat().st_size for f in model_files)
    print(f"\n💾 Model size: {total_size / 1024 / 1024:.1f} MB")

print(f"\n📝 Architecture comparison:")
print(f"  Tiny model:       {results['model_params']:,} parameters,   32 dim,  1 layer,  2 heads")
print(f"  Production model: {production_results['model_params']:,} parameters, 128 dim, 6 layers, 8 heads")
improvement = production_results['model_params'] / results['model_params']
print(f"  Improvement:      {improvement:.0f}x more parameters!")

print(f"\n📁 Production model saved to: {production_trainer.output_dir}")

# Store the production model path for later use
production_model_path = production_trainer.output_dir


🚀 Training production CalcGPT model...
⚠️ This will take longer but results in much better performance!


KeyboardInterrupt: 

## 🎉 Part 5: Production Model Evaluation

### Comprehensive Testing

Now let's evaluate our production model and compare it to the tiny model. We expect to see dramatic improvements across all metrics.

#### What to Look For

1. **Higher Accuracy**: Better arithmetic correctness
2. **Better Generalization**: Performance on unseen number combinations  
3. **Format Consistency**: More reliable expression formatting
4. **Faster Convergence**: Stable performance across test types

Let's run the comprehensive evaluation suite!


In [ ]:
# Comprehensive evaluation of production CalcGPT using the library
print("📊 Evaluating production CalcGPT model...")
print("🎯 This will test the model on diverse arithmetic problems")

# Create comprehensive evaluation configuration
production_eval_config = EvaluationConfig(
    sample_size=200,         # Test on 200 random cases
    max_tokens=15,           # Allow more tokens for complex expressions
    verbose=True
)

# Run comprehensive evaluation
eval_start = time.time()

production_evaluator = CalcGPTEvaluator(
    config=production_eval_config,
    model_path=production_model_path,      # Use our production model
    dataset_path=production_dataset,       # Use production dataset
    verbose=True
)

production_eval_results = production_evaluator.evaluate()
eval_time = time.time() - eval_start

print(f"\n📊 Production Evaluation Results:")
print(f"  🎯 Overall accuracy: {production_eval_results['accuracy_stats']['overall']:.1%}")
print(f"  ✅ Format validity: {production_eval_results['accuracy_stats']['format']:.1%}")
print(f"  🧮 Arithmetic correctness: {production_eval_results['accuracy_stats']['arithmetic']:.1%}")
print(f"  📝 Complete expressions: {production_eval_results['accuracy_stats']['complete']:.1%}")

print(f"\n📈 Performance by Test Type:")
for test_type, stats in production_eval_results['test_type_stats'].items():
    print(f"  {test_type.replace('_', ' ').title()}:")
    print(f"    Arithmetic: {stats['arithmetic']:.1%}")
    print(f"    Format: {stats['format']:.1%}")

print(f"\n⏱️ Performance Timing:")
timing = production_eval_results['timing_stats']
print(f"  Mean: {timing['mean']:.1f}ms")
print(f"  Median: {timing['median']:.1f}ms")
print(f"  Range: {timing['min']:.1f}ms - {timing['max']:.1f}ms")

print(f"\n⏱️ Evaluation completed in {eval_time:.1f} seconds")

# Test on specific challenging problems using programmatic interface
print("\n" + "="*60)
print("🧠 CHALLENGING ARITHMETIC TESTS")
print("="*60)

# Initialize production inference model
production_inference_config = InferenceConfig(
    temperature=0.0,         # Deterministic inference
    max_tokens=15,
    verbose=False
)

production_calc_model = CalcGPT(
    config=production_inference_config,
    model_path=production_model_path,
    verbose=False
)

challenging_problems = [
    "99+1",      # Near boundary
    "100-50",    # Large subtraction  
    "50+50",     # Equal operands
    "0+100",     # Edge cases
    "100-100",   # Zero result
    "85+15",     # Carry operations
    "73-28",     # Complex subtraction
    "42+37",     # Mid-range addition
]

print("Testing production model on challenging problems:")
print("Problem       → Answer   (Expected)  Status")
print("-" * 50)

correct_count = 0
for problem in challenging_problems:
    try:
        result = production_calc_model.generate(problem + "=")
        predicted_answer = result['completion'].strip()
        
        # Calculate expected answer
        if '+' in problem:
            operands = problem.split('+')
            expected = int(operands[0]) + int(operands[1])
        elif '-' in problem:
            operands = problem.split('-')
            expected = int(operands[0]) - int(operands[1])
        else:
            expected = "?"
        
        # Check correctness
        is_correct = str(predicted_answer) == str(expected)
        status = "✅ CORRECT" if is_correct else "❌ WRONG"
        if is_correct:
            correct_count += 1
        
        print(f"{problem:12s} → {predicted_answer:8s} ({expected:8s})  {status}")
        
    except Exception as e:
        print(f"{problem:12s} → ERROR     (?)        ❌")

accuracy = correct_count / len(challenging_problems) * 100
print(f"\n🎯 Challenge Test Accuracy: {correct_count}/{len(challenging_problems)} ({accuracy:.1f}%)")

# Compare with tiny model
print(f"\n📊 Model Comparison:")
print(f"  Tiny model accuracy:       {eval_results['accuracy_stats']['overall']:.1%}")
print(f"  Production model accuracy: {production_eval_results['accuracy_stats']['overall']:.1%}")
improvement = production_eval_results['accuracy_stats']['overall'] - eval_results['accuracy_stats']['overall']
print(f"  Improvement:               +{improvement:.1%}")

if accuracy >= 90:
    print("\n🏆 EXCELLENT! Production model shows strong arithmetic capabilities!")
elif accuracy >= 70:
    print("\n👍 GOOD! Model demonstrates solid arithmetic understanding!")
elif accuracy >= 50:
    print("\n📈 MODERATE! Model shows some arithmetic capability but needs improvement!")
else:
    print("\n⚠️ NEEDS WORK! Consider additional training or architectural changes!")


## 🎮 Part 6: Interactive Usage & Deployment

### Production-Ready Inference

Our CalcGPT model is now ready for real-world usage! The CalcGPT CLI provides multiple interfaces:

#### 🖥️ Interactive Mode
```bash
python calcgpt.py -i
# Provides a beautiful interactive calculator interface
```

#### 📦 Batch Processing  
```bash
python calcgpt.py -b "50+50" "99-1" "75+25"
# Process multiple problems at once
```

#### 📄 File Processing
```bash
echo "100+1\n50+50\n99-99" > problems.txt
python calcgpt.py -f problems.txt -o results.json
```

### Model Analysis & Introspection

Our intelligent naming system allows easy model analysis:

```bash
python calcgpt_train.py --analyze models/calcgpt_emb128_lay6_head8_ep20_bs8_lr1e3_dsm100
# Shows complete training configuration and equivalent command
```

Let's demonstrate the interactive capabilities!


In [ ]:
# Demonstrate various CalcGPT usage modes
print("🎮 CalcGPT Usage Demonstrations")
print("="*50)

# 1. Programmatic batch processing
print("\n1️⃣ Programmatic Batch Processing")
batch_problems = ["25+25=", "100-33=", "67+12=", "88-44=", "75+20="]

print(f"Input problems: {[p.replace('=', '') for p in batch_problems]}")
print("Results using CalcGPT library:")

batch_inference_config = InferenceConfig(temperature=0.0, verbose=False)
batch_calc_model = CalcGPT(
    config=batch_inference_config,
    model_path=production_model_path,
    verbose=False
)

correct_count = 0
for problem in batch_problems:
    try:
        result = batch_calc_model.generate(problem)
        predicted = result['completion'].strip()
        
        # Calculate expected
        expr = problem.replace('=', '')
        if '+' in expr:
            operands = expr.split('+')
            expected = int(operands[0]) + int(operands[1])
        elif '-' in expr:
            operands = expr.split('-')
            expected = int(operands[0]) - int(operands[1])
        
        is_correct = str(predicted) == str(expected)
        status = "✅" if is_correct else "❌"
        if is_correct:
            correct_count += 1
        
        print(f"  {expr} → {predicted} {status}")
        
    except Exception as e:
        print(f"  {expr} → ERROR ❌")

print(f"Accuracy: {correct_count}/{len(batch_problems)} ({correct_count/len(batch_problems)*100:.1f}%)")

# 2. CLI batch processing with JSON output (demonstrating the CLI tool)
print("\n2️⃣ CLI Batch Processing with JSON Output")
result = subprocess.run([
    'python', 'calcgpt.py',
    '-b', '25+25', '100-33', '67+12', '88-44', '75+20',
    '--format', 'json',
    '--no-banner'
], capture_output=True, text=True)

if result.stdout:
    try:
        output_data = json.loads(result.stdout)
        print(f"CLI Results - {output_data['metadata']['correct_answers']}/{output_data['metadata']['total_problems']} correct")
        for res in output_data['results']:
            status = "✅" if not res.get('error') else "❌"
            print(f"  {res['problem']} → {res.get('answer', 'ERROR')} {status}")
    except:
        print("Raw output:", result.stdout)

# 3. Performance comparison: Tiny vs Production (programmatic)
print("\n3️⃣ Performance Comparison: Tiny vs Production")

comparison_problems = ["1+1=", "10+5=", "25+25=", "50-20=", "99+1="]

# Initialize tiny model for comparison
tiny_calc_model = CalcGPT(
    config=InferenceConfig(temperature=0.0, verbose=False),
    model_path=tiny_model_path,
    verbose=False
)

print("Problem   | Tiny Model  | Production Model | Better?")
print("-" * 55)

for problem in comparison_problems:
    expr = problem.replace('=', '')
    
    # Get expected answer
    if '+' in expr:
        operands = expr.split('+')
        expected = int(operands[0]) + int(operands[1])
    elif '-' in expr:
        operands = expr.split('-')
        expected = int(operands[0]) - int(operands[1])
    
    # Test tiny model
    try:
        tiny_result = tiny_calc_model.generate(problem)
        tiny_answer = tiny_result['completion'].strip()
        tiny_correct = str(tiny_answer) == str(expected)
    except:
        tiny_answer = "ERROR"
        tiny_correct = False
    
    # Test production model
    try:
        prod_result = production_calc_model.generate(problem)
        prod_answer = prod_result['completion'].strip()
        prod_correct = str(prod_answer) == str(expected)
    except:
        prod_answer = "ERROR"
        prod_correct = False
    
    # Determine which is better
    if prod_correct and not tiny_correct:
        better = "🚀 YES"
    elif prod_correct and tiny_correct:
        better = "✅ BOTH"
    elif not prod_correct and tiny_correct:
        better = "🤔 TINY"
    else:
        better = "❌ NONE"
    
    tiny_status = "✅" if tiny_correct else "❌"
    prod_status = "✅" if prod_correct else "❌"
    
    print(f"{expr:8s}  | {tiny_answer:6s} {tiny_status:2s} | {prod_answer:10s} {prod_status:2s}     | {better}")

# 4. CLI advanced features demonstration
print("\n4️⃣ CLI Advanced Features - Temperature Control")

print("🎯 Temperature Control (CLI demonstration):")
test_problem = "50+50"

for temp in [0.0, 0.5, 1.0]:
    result = subprocess.run([
        'python', 'calcgpt.py',
        '-b', test_problem,
        '--temperature', str(temp),
        '--no-banner'
    ], capture_output=True, text=True)
    
    # Extract answer
    answer = "ERROR"
    for line in result.stdout.split('\n'):
        if "50+50" in line:
            parts = line.split()
            if len(parts) >= 2:
                answer = parts[1]
                break
    
    randomness = "deterministic" if temp == 0.0 else f"randomness={temp}"
    print(f"  Temperature {temp}: {test_problem} → {answer} ({randomness})")

print(f"\n🎉 CalcGPT Library & CLI Tools Demonstrated!")
print(f"   ✅ Programmatic access via lib/ package")
print(f"   ✅ CLI tools for interactive usage")  
print(f"   ✅ Multiple input/output formats")
print(f"   ✅ Model comparison capabilities")
print(f"   ✅ Professional evaluation tools")


## 🎓 Part 7: Lessons Learned & Advanced Concepts

### 🧠 Key Insights from Building CalcGPT

Through this journey, we've learned fundamental principles that apply to all transformer-based language models:

#### 1. **Architecture Scaling Laws**
- **Parameters matter**: 30x more parameters → dramatically better performance
- **Depth vs Width**: More layers often better than wider layers
- **Attention heads**: Multiple heads capture different relationships
- **Context length**: Longer sequences enable more complex reasoning

#### 2. **Data Engineering Principles**  
- **Quality over quantity**: Clean, systematic data beats noisy large datasets
- **Data augmentation**: Simple transformations (like commutativity) boost performance
- **Distribution coverage**: Ensure training data covers the inference domain
- **Intelligent naming**: Systematic dataset organization enables reproducibility

#### 3. **Training Dynamics**
- **Learning rate scheduling**: Cosine annealing provides smooth convergence
- **Validation monitoring**: Early stopping prevents overfitting
- **Batch size trade-offs**: Larger batches for stability, smaller for regularization
- **Mixed precision**: Significant speedups with minimal accuracy loss

#### 4. **Evaluation Methodologies**
- **Multiple test types**: Different completion scenarios reveal different capabilities
- **Comprehensive metrics**: Format, correctness, and performance matter
- **Generalization testing**: Test beyond training distribution
- **Error analysis**: Understanding failures guides improvements

### 🔬 What Makes CalcGPT Special?

Unlike general language models that struggle with arithmetic, CalcGPT demonstrates:

- **Precise computation**: Exact arithmetic rather than approximate pattern matching
- **Systematic reasoning**: Step-by-step problem solving
- **Format consistency**: Reliable output structure
- **Scalable performance**: Handles increasing complexity gracefully

### 🚀 Advanced Concepts & Extensions

Ready to take CalcGPT further? Here are some advanced directions:

#### 🧮 Extended Arithmetic
- **Multiplication & Division**: More complex operations
- **Multi-step problems**: (a+b)×c, nested operations
- **Decimal numbers**: Floating-point arithmetic
- **Negative numbers**: Full integer arithmetic

#### 🏗️ Architectural Improvements  
- **Positional encodings**: Learned vs sinusoidal
- **Attention mechanisms**: Sparse attention, local attention
- **Normalization strategies**: LayerNorm vs RMSNorm
- **Activation functions**: ReLU vs GELU vs SwiGLU

#### 📊 Training Enhancements
- **Curriculum learning**: Start simple, gradually increase complexity
- **Data mixing**: Combine arithmetic with natural language
- **Multi-task learning**: Multiple mathematical operations simultaneously
- **Reinforcement learning**: Self-improvement through interaction

#### 🔧 Production Optimizations
- **Model quantization**: 8-bit or 4-bit inference
- **Knowledge distillation**: Smaller models from larger ones
- **Caching strategies**: KV-cache optimization
- **Batch processing**: Efficient multi-query handling


## 🌟 Summary & Next Steps

### 🎯 What We Accomplished

In this comprehensive tutorial, we built a complete machine learning system from scratch:

#### 🛠️ **Tools Created**
- **CalcGPT DataGen**: Intelligent dataset generation with parameter encoding
- **CalcGPT Trainer**: Professional training system with auto-naming
- **CalcGPT Eval**: Comprehensive evaluation and analysis
- **CalcGPT CLI**: Production-ready inference interface

#### 📊 **Models Trained**
- **Tiny Model**: 38K parameters, proof of concept (0-5 arithmetic)
- **Production Model**: 1.2M parameters, real-world capable (0-100 arithmetic)

#### 🧠 **Core Concepts Mastered**
- Transformer architecture and attention mechanisms
- Character-level language modeling for arithmetic
- Dataset engineering and augmentation strategies  
- Training dynamics and optimization techniques
- Comprehensive evaluation methodologies
- Production deployment and model management

### 🚀 Your Learning Journey Continues

#### **Immediate Next Steps**
1. **Experiment**: Try different model architectures and training settings
2. **Extend**: Add multiplication, division, or decimal arithmetic
3. **Scale**: Train on larger datasets with higher number ranges
4. **Deploy**: Use CalcGPT in real applications or integrate via API

#### **Advanced Projects**
- **Multi-modal**: Combine text and visual arithmetic problems
- **Interactive Tutoring**: Build an AI math tutor
- **Scientific Computing**: Extend to algebraic expressions
- **Model Optimization**: Quantization and efficient inference

### 📚 Additional Resources

#### **HuggingFace & Transformers**
- [Transformers Documentation](https://huggingface.co/docs/transformers)
- [Course: NLP with Transformers](https://huggingface.co/course)
- [Model Hub](https://huggingface.co/models)

#### **PyTorch Deep Learning**
- [PyTorch Tutorials](https://pytorch.org/tutorials)
- [Deep Learning with PyTorch](https://pytorch.org/deep-learning-with-pytorch)

#### **Research Papers**
- [Attention Is All You Need](https://arxiv.org/abs/1706.03762) (Original Transformer)
- [Language Models are Few-Shot Learners](https://arxiv.org/abs/2005.14165) (GPT-3)
- [Training Compute-Optimal Large Language Models](https://arxiv.org/abs/2203.15556) (Scaling Laws)

### 🎉 Congratulations!

You've successfully built a complete transformer-based language model system! You now understand:

- ✅ How transformers work under the hood
- ✅ Professional ML engineering practices  
- ✅ Dataset design and evaluation strategies
- ✅ Production deployment considerations
- ✅ The full ML lifecycle from data to deployment

**Keep experimenting, keep learning, and keep building amazing AI systems!** 🚀

---

*Built with ❤️ using CalcGPT - A comprehensive transformer tutorial by Mihai NADAS*
